In [76]:
#import neccessary modules
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [77]:
# helper functions
from timeseries_helpers import datasetstorer
from timeseries_helpers import dataset_importer
from timeseries_helpers import database_importer
from timeseries_helpers import standardizer
from data_preprocessing import extract_accelerometer_data, csv_to_dataset_list

In [78]:
ds = csv_to_dataset_list('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gerade.csv')

    



In [79]:
len(ds)

In [80]:
# converts the given list of dataframes to one dataframe
df_list = []
for idx,e in enumerate(ds):
    df = e.assign(punch_id=idx)
    df['label'] = 0
    df_list.append(df)
df_res = pd.concat(df_list)
df_new = df_res.reset_index(drop=True)
df_new = df_new.rename(index=str, columns={"accelerometerAccelerationX(G)": "a_x", "accelerometerAccelerationY(G)": "a_y", "accelerometerAccelerationZ(G)":"a_z"})

In [81]:
len(df_new)

In [82]:
df_new.head()